In [86]:
import pandas as pd 
import networkx as nx
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
import pandas as pd
import xgboost
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split

In [84]:
att = np.array(pd.read_csv('data/attributes.csv'))
att

array([[0, 'l'],
       [1, 'x'],
       [2, 'x'],
       ...,
       [1497, 'l'],
       [1498, 'f'],
       [1499, 'l']], dtype=object)

In [77]:
G = nx.read_edgelist('data/edges_train.edgelist', data=False, delimiter=',', nodetype=int)
# nx.set_node_attributes(G, att, 'community')

# nx.draw(G, node_size=50, width=0.2)

In [78]:
G.number_of_nodes(), G.number_of_edges()

(1500, 6600)

#### setting:
we have an edgelist containing 6600 links of 1500 nodes and an attribute list containing 1500 nodes with a corresponding level of a categorical attribute variable.
#### aim:
predict the missing links -> should amount to being 7333 links

# Implementation of all possible Link Prediction metrics from networkx
ADD ALL NEW FEATURES TO GETFEATURES

In [115]:
# Input: getFeature(graph, node_i, node_j)

communities = list(nx.community.greedy_modularity_communities(G))

def getFeatures(G, i, j, comms):
    # ressource allocation index
    ra = list(nx.resource_allocation_index(G, [(i, j)]))[0][2]
    
    # # jaccard coefficient
    # jc = list(nx.jaccard_coefficient(G, [(i, j)]))[0][2]
    
    # # adamic adar index
    # aa = list(nx.adamic_adar_index(G, [(i, j)]))[0][2]
    
    # preferential attachment
    pa = list(nx.preferential_attachment(G, [(i, j)]))[0][2]
    
    # #common neighbors soundarajan hopcroft
    # sh = list(nx.cn_soundarajan_hopcroft(G, [(i, j)]))[0][2]

    # #ra index soundarajan hopcroft
    # rai = list(nx.ra_index_soundarajan_hopcroft(G, [(i, j)]))[0][2]

    # #within inter cluster
    # wic = list(nx.within_inter_cluster(G, [(i, j)]))[0][2]

    # amount of common neighbors
    cn = len(list(nx.common_neighbors(G, i, j)))

    # check if the nodes are in the same cluster/same attribute
    att_same = 1 if att[i][1] == att[j][1] else 0

    # nx.shortest_path_length(G, i, j)

    #node based features -> could be worth a try
    # degree_i = G.degree(i)
    # degree_j = G.degree(j)
    clustering_coeff_i = nx.clustering(G, i)
    clustering_coeff_j = nx.clustering(G, j)
    
    # betweenness_centrality_i = nx.betweenness_centrality(G)[i]
    # betweenness_centrality_j = nx.betweenness_centrality(G)[j]

    # Edge betweenness centrality
    # edge_betweenness = nx.edge_betweenness_centrality(G)[(i, j)]


    katz_centrality_i = nx.katz_centrality_numpy(G)[i]
    katz_centrality_j = nx.katz_centrality_numpy(G)[j]

    # avg_neighbor_degree_i = nx.average_neighbor_degree(G)[i]
    # avg_neighbor_degree_j = nx.average_neighbor_degree(G)[j]

    # closeness_centrality_i = nx.closeness_centrality(G, i)
    # closeness_centrality_j = nx.closeness_centrality(G, j)

    # pagerank_i = nx.pagerank(G)[i]
    # pagerank_j = nx.pagerank(G)[j]


    same_community = 1 if any(i in comm and j in comm for comm in comms) else 0


    # Add the new features to the return list
    return [ra, pa, cn, att_same, clustering_coeff_i, clustering_coeff_j,
            katz_centrality_i, katz_centrality_j,
            same_community,
            i, j
            ]


In [87]:
# Compute HITS scores for the graph
if not os.path.isfile('data/hits.p'):
    hits = nx.hits(G, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits, open('data/hits.p', 'wb'))
else:
    hits = pickle.load(open('data/hits.p', 'rb'))

def getFeatures2(G, i, j, comms, hits):
    # Extract the most important features based on previous analysis
    pa = list(nx.preferential_attachment(G, [(i, j)]))[0][2]
    cn = len(list(nx.common_neighbors(G, i, j)))
    att_same = 1 if att[i][1] == att[j][1] else 0
    clustering_coeff_i = nx.clustering(G, i)
    clustering_coeff_j = nx.clustering(G, j)
    katz_centrality_i = nx.katz_centrality_numpy(G)[i]
    katz_centrality_j = nx.katz_centrality_numpy(G)[j]
    same_community = 1 if any(i in comm and j in comm for comm in comms) else 0

    # HITS scores
    authority_i = hits[1][i]
    authority_j = hits[1][j]
    hub_i = hits[0][i]
    hub_j = hits[0][j]

    return [pa, cn, att_same, clustering_coeff_i, clustering_coeff_j,
            katz_centrality_i, katz_centrality_j, authority_i, authority_j, 
            hub_i, hub_j,same_community]

In [88]:
# check if att_same works
att[2][1] == att[0][1]

False

the idea is to create features for all current 

In [116]:
X = []
y = []


for (i, j) in G.edges:
    X.append(getFeatures(G, i, j, communities))
    y.append(1)
    print(i,j)
solInput = pd.read_csv('data/solutionInput.csv')

# set length of 0s to modify ratio (currently set to 1:1)
length_to_modify_ratio = len(X)

for kk in range(length_to_modify_ratio):
    #set possible i and j ranges
    possibleiandj = len(att)

    i = np.random.randint(possibleiandj)
    j = np.random.randint(possibleiandj)
    print("kk" + str(kk))
    # check if edge already exists -> should be yes
    # check if i and j are the same -> no edge to itself
    # check if i and j are in the solution set/x_test -> should not be set to anything to avoid distorting the solution
    while (i, j) in G.edges or i == j or any((i == row['int1'] and j == row['int2']) or (i == row['int2'] and j == row['int1']) for _, row in solInput.iterrows()):
        i = np.random.randint(possibleiandj)
        j = np.random.randint(possibleiandj)
    X.append(getFeatures(G, i, j, communities))
    y.append(0)

0 5
0 6
0 7
0 8
0 9
0 12
0 13
0 14
0 16
0 17
0 18
0 21
0 22
0 23
0 26
0 34
0 35
0 40
0 48
0 55
0 57
0 63
0 71
0 81
0 103
0 113
0 115
0 118
0 120
0 124
0 130
0 161
0 162
0 173
0 177
0 178
0 179
0 204
0 213
0 232
0 239
0 242
0 247
0 112
0 218
0 76
0 102
0 323
5 2
5 3
5 4
5 6
5 7
5 8
5 9
5 10
5 12
5 13
5 15
5 17
5 20
5 22
5 28
5 32
5 35
5 36
5 43
5 47
5 53
5 60
5 61
5 70
5 75
5 76
5 77
5 83
5 85
5 86
5 90
5 94
5 97
5 98
5 100
5 101
5 107
5 108
5 113
5 114
5 115
5 116
5 119
5 125
5 126
5 133
5 135
5 139
5 147
5 153
5 162
5 164
5 165
5 174
5 186
5 201
5 203
5 232
5 239
5 241
5 23
5 561
5 521
5 266
5 769
6 1
6 4
6 11
6 15
6 38
6 40
6 42
6 70
6 120
6 122
6 150
6 168
6 202
6 288
6 190
6 1105
6 668
6 858
7 2
7 3
7 4
7 8
7 9
7 11
7 14
7 20
7 21
7 23
7 26
7 27
7 34
7 38
7 39
7 42
7 50
7 52
7 53
7 55
7 57
7 58
7 61
7 66
7 69
7 75
7 77
7 78
7 82
7 89
7 97
7 102
7 110
7 113
7 119
7 124
7 125
7 126
7 131
7 132
7 134
7 136
7 145
7 146
7 152
7 154
7 159
7 168
7 172
7 177
7 179
7 182
7 183
7 189
7 193
7

In [129]:
# X = []
# y = []


# for (i, j) in G.edges:
#     X.append(getFeatures2(G, i, j, communities, hits))
#     y.append(1)
#     print(i,j)
# solInput = pd.read_csv('data/solutionInput.csv')

# # set length of 0s to modify ratio (currently set to 1:1)
# length_to_modify_ratio = len(X)

# for kk in range(length_to_modify_ratio):
#     #set possible i and j ranges
#     possibleiandj = len(att)

#     i = np.random.randint(possibleiandj)
#     j = np.random.randint(possibleiandj)
#     print("kk" + str(kk))
#     # check if edge already exists -> should be yes
#     # check if i and j are the same -> no edge to itself
#     # check if i and j are in the solution set/x_test -> should not be set to anything to avoid distorting the solution
#     while (i, j) in G.edges or i == j or any((i == row['int1'] and j == row['int2']) or (i == row['int2'] and j == row['int1']) for _, row in solInput.iterrows()):
#         i = np.random.randint(possibleiandj)
#         j = np.random.randint(possibleiandj)
#     X.append(getFeatures2(G, i, j, communities, hits))
#     y.append(0)

In [163]:
num_rows = len(X)
num_cols = len(X[0]) if X else 0
print(f"Dimensions of features: {num_rows} rows, {num_cols} columns")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [134]:
X_train.shape

(10560, 12)

In [119]:
X_kaggle = []
#X_kaggle_2 = []
X_kaggle = [getFeatures(G, i, j, communities) for i, j in zip(solInput['int1'], solInput['int2'])]
#X_kaggle_2 = [getFeatures2(G, i, j, communities, hits) for i, j in zip(solInput['int1'], solInput['int2'])]